***WMC via Knowledge compilation***


---


1. Write a method that transforms a formula in sd-DNNF form, and use
this method for computing the weighted model count of propositional
formulas.

2. Check the correctness of your algorithm by comparing the results of
your method with the explicit computation of weighted model
counting via truth table.

3. Write a method that estimates #SAT using the sampleSat algorithm,
and compare the result of the approximated counting with the result
obtained by the exact counting.






In [114]:
#import libraries
from sympy import symbols, simplify, Not, And, Or, Implies, satisfiable, S, simplify, sympify
from sympy import symbols, And, Or, Not, Implies
from sympy.logic.boolalg import to_nnf, Boolean
from sympy.logic.inference import satisfiable
from sympy.logic.boolalg import to_cnf
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from itertools import combinations
from functools import reduce
import random
import time

In [115]:
from IPython.display import display, HTML

def render_pipeline(stages):
    # stages: list[(name, done)]
    badges = []
    for name, done in stages:
        icon = "✅" if done else "⏳"
        opacity = "1" if done else ".6"
        badges.append(f"""
        <div style="display:flex; align-items:center; gap:8px; padding:8px 12px;
                    border:1px solid #e5e7eb; border-radius:999px; background:#ffffff; margin:6px;
                    font-family:Inter, Arial; opacity:{opacity};">
            <span style="width:10px; height:10px; border-radius:999px; background:{'#10b981' if done else '#fbbf24'}; display:inline-block;"></span>
            <span style="font-size:14px;">{icon}</span>
            <span style="font-weight:600; font-size:14px;">{name}</span>
        </div>""")
    n_done = sum(1 for _, d in stages if d)
    total = len(stages) or 1
    pct = int(100 * n_done / total)
    bar = f"""
    <div style="height:10px; background:#f0f0f0; border-radius:6px; overflow:hidden; margin-top:6px;">
        <div style="height:10px; width:{pct}%; background:#10b981;"></div>
    </div>"""
    return f"""
    <div>
        <div style="display:flex; flex-wrap:wrap;">{''.join(badges)}</div>
        <div style="font-family:Inter, Arial; margin:6px 2px; font-size:13px;">Progress: {pct}%</div>
        {bar}
    </div>"""

In [116]:
from IPython.display import display, HTML

def render_kpi_cards(results):
    # results: list of dicts with keys: name, exact, approx
    cards = []
    for r in results:
        exact = float(r.get("exact", 0))
        calculated = float(r.get("calculated", 0))
        delta = calculated - exact
        rel = (abs(delta) / exact) if exact != 0 else float("inf")

        def fmt_int(x):
            try:
                x = int(x)
                return f"{x:,}".replace(",", ".")
            except Exception:
                return str(x)

        def fmt_float(x):
            try:
                return f"{x:.4f}"
            except Exception:
                return str(x)

        cards.append(f"""
        <div style="flex:1; min-width:260px; max-width:360px;
                    border:1px solid #e0dede; border-radius:18px; padding:18px; margin:12px;
                    background:linear-gradient(135deg, #fef6f9, #f3f8ff);
                    font-family:Inter, Arial, sans-serif; color:#333; 
                    box-shadow:0 4px 10px rgba(0,0,0,0.05);">
            <div style="font-size:14px; color:#666; font-weight:500;">Test</div>
            <div style="font-size:20px; font-weight:700; margin-bottom:12px; color:#444;">
                {r.get('name','N/A')}
            </div>
            <div style="font-size:14px; color:#666; font-weight:500;">Formula</div>
            <div style="font-size:20px; font-weight:700; margin-bottom:12px; color:#444;">
                {r.get('formula','N/A')}
            </div>
            <div style="font-size:14px; color:#666; font-weight:500;">sd-DNNF</div>
            <div style="font-size:20px; font-weight:700; margin-bottom:12px; color:#444;">
                {r.get('sd-DNNF','N/A')}
            </div>
            <div style="display:flex; gap:14px; align-items:baseline; margin:6px 0;">
                <div style="flex:1; background:#d9f2e6; color:#224; border-radius:12px; padding:10px;">
                    <div style="font-size:12px; opacity:.8;">Exact</div>
                    <div style="font-size:20px; font-weight:700;">{fmt_int(exact)}</div>
                </div>
                <div style="flex:1; background:#ffe6ea; color:#224; border-radius:12px; padding:10px;">
                    <div style="font-size:12px; opacity:.8;">Calculated</div>
                    <div style="font-size:20px; font-weight:700;">{fmt_int(calculated)}</div>
                </div>
            </div>
            <div style="display:flex; gap:14px; margin-top:12px;">
                <div style="flex:1; border:1px dashed #ccc; border-radius:12px; padding:10px; background:#f7fdfb;">
                    <div style="font-size:12px; color:#666;">Δ (calculated - exact)</div>
                    <div style="font-weight:700; color:#333;">{fmt_int(delta)}</div>
                </div>
            </div>
        </div>
        """)

    html = f"""
    <div style="display:flex; flex-wrap:wrap; gap:10px; align-items:stretch;">
        {''.join(cards)}
    </div>
    """
    display(HTML(html))

In [117]:
# ===========================
# Styling pastel
# ===========================
def style_pastel(df: pd.DataFrame):
    df = df.drop(columns=['relative_error','p_hat','n'], errors='ignore')
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    sty = (df.style
           .hide(axis="index")
           .set_table_styles([
               {"selector": "th.col_heading",
                "props": [("background", "#f0f0f0"), ("color", "#374151"),
                          ("font-size", "13px"), ("font-weight", "600"),
                          ("padding", "10px 8px"), ("border", "none")]},
               {"selector": "tbody td",
                "props": [("padding", "10px 12px"), ("border-bottom", "1px solid #e5e7eb"),
                          ("font-size", "13px"), ("color", "#f0f0f0")]},
               {"selector": "table",
                "props": [("border-collapse", "separate"), ("border-spacing", "0"),
                          ("border", "1px solid #e5e7eb"), ("border-radius", "12px"),
                          ("overflow", "hidden"), ("box-shadow", "0 4px 12px rgba(0,0,0,.05)")]},
           ])
           .format(precision=4)
          )

    if num_cols:
        # Gradiente pastello (viola→rosa→azzurro chiaro)
        sty = sty.background_gradient(
            cmap="coolwarm", subset=num_cols,
            vmin=df[num_cols].min().min(),
            vmax=df[num_cols].max().max()
        )

    return sty

Pipeline to convert a formula F into sd-DNNF:
1.	Start with a formula **F**.
2.	Transform it into ***NNF*** (Negation Normal Form): a formula where negations appear only in front of atomic propositions.
3.	Transform it into ***DNNF*** (Decomposable Negation Normal Form): an NNF where each conjunction shares no atomic propositions, i.e., Φ₁ ∧ Φ₂ such that props(Φ₁) ∩ props(Φ₂) = ∅. To achieve this, Shannon’s Expansion is applied.
4.	Transform it into ***d-DNNF*** (Deterministic Decomposable Negation Normal Form): a DNNF where, for every disjunction Φ₁ ∨ Φ₂ ∨ … ∨ Φₙ in the formula, there is at most one i such that I |= Φᵢ.
This ensures that the branches of an OR are mutually exclusive; otherwise, in Weighted Model Counting (WMC), we risk double-counting the same truth assignment.
5.	Transform it into ***sd-DNNF*** (Smooth Deterministic Decomposable Negation Normal Form): apply smoothing (left and right) to guarantee that all disjuncts contain all propositional variables — see the transformation rules (in practice, a tautology is introduced for missing atomic propositions).

Note: Strictly speaking, the very first step would be the transformation of F into CNF (Conjunctive Normal Form): a formula in which implication (→) and equivalence (≡) symbols do not appear. However, in our code this step is handled by the to_nnf function (imported).

In [118]:
# ============================================================
#  F -> NNF -> DNNF -> d-DNNF -> sd-DNNF (con Tau) -> WMC
#  (no normalize_formula: use directly to_nnf)
# ============================================================
# ---------------------------
# Boolean node Tau(v) for visible structural smoothing
# ---------------------------
class Tau(Boolean):
    """
    Represents the tautology (v ∨ ¬v) as an explicit node.
    In WMC, it contributes [w(v) + w(¬v)].
    """
    is_Tau = True

    def __new__(cls, v):
        v = sympify(v)
        obj = Boolean.__new__(cls)
        obj._v = v
        return obj

    @property
    def v(self):
        return self._v

    @property
    def args(self):
        return (self._v,)

    def _hashable_content(self):
        return (self._v,)

    @property
    def free_symbols(self):
        return {self._v}

def _tau(v):
    return Tau(v)

def _vars(e):
    return set(e.free_symbols)

# ---------------------------
# Shannon expansion (NO simplify)
# ---------------------------
def shannon_expansion(f, s):
    """
    f == (s ∧ f|s=True) ∨ (¬s ∧ f|s=False)
    """
    if s not in f.free_symbols:
        return f
    f1 = f.subs({s: S.true})
    f0 = f.subs({s: S.false})
    return Or(And(s, f1), And(Not(s), f0))

# ---------------------------
# Utilities: variable-connected components
# ---------------------------
def _var_components(conj):
    """
    Group the factors of an AND into variable-connected blocks.
    Different blocks do not share variables -> decomposable AND.
    """
    if isinstance(conj, And):
        factors = list(conj.args)
    else:
        factors = [conj]
    comps = []
    while factors:
        seed = factors.pop()
        group = [seed]
        vars_grp = set(seed.free_symbols)
        changed = True
        while changed:
            changed = False
            rest = []
            for f in factors:
                if vars_grp & set(f.free_symbols):
                    group.append(f)
                    vars_grp |= set(f.free_symbols)
                    changed = True
                else:
                    rest.append(f)
            factors = rest
        comps.append(group)
    return comps

# ---------------------------
# 1) NNF -> DNNF
# Function that takes a NNF and returns a DNNF.
# It applies Shannon's Expansion in order to ensure decomposability.
# Every conjunction does not have common atomic propositions, i.e. Φ₁ ∧ Φ₂ such that props(Φ₁) ∩ (Φ₂) = ∅.
# ---------------------------
def NNF2DNNF(nnf):

    #Take Boolean variables from the formula and put them in a list
    #nnf.free_symbols is a set of SymPy symbols that appear in the expression
    atoms = list(nnf.free_symbols)

    #base case: the formula has 0 or 1 variable
    if len(atoms) <= 1:
        return simplify(nnf)

    #I build groups that are “variable-connected”: if I have only one group → not decomposable ⇒ I apply Shannon
    if isinstance(nnf, And): #check if current node is conjunction
        # _var_components takes the factors of the AND (e.g., F1 ∧ F2 ∧ F3) and groups 
        # Them into blocks such that within each block the factors share variables. 
        # If different blocks do not share variables → they are candidates for a decomposable AND.
        # Example: (A∨B)∧(¬A∨C)∧(D∨E) produces groups = [[A∨B, ¬A∨C], [D∨E]].
        groups = _var_components(nnf)
        if len(groups) > 1:
            compiled = [
                NNF2DNNF(And(*g)) if len(g) > 1 else NNF2DNNF(g[0])
                for g in groups ]
            return simplify(And(*compiled))  # decomposable AND 
        else: # No possible decomposition -> force split with Shannon
            pivot = atoms[0]
            return NNF2DNNF(shannon_expansion(nnf, pivot))

    if isinstance(nnf, Or):
        kids = [NNF2DNNF(k) for k in nnf.args]
        return simplify(Or(*kids))          # determinism after
    
    # fallback
    pivot = atoms[0]
    return NNF2DNNF(shannon_expansion(nnf, pivot))

# ---------------------------
# 2) DNNF -> d-DNNF (determinism on OR)
# ---------------------------
def _make_or_deterministic(expr):
    """
    Makes Or(...) deterministic without infinite recursions.
    Strategy:
    - as long as there is a joinable pair of children,
    choose a pivot in the INTERSECTION of their variables (if empty, in the UNION),
    apply Shannon to the entire OR and restart. Iterative.
    """
    assert isinstance(expr, Or)
    # flatten + dedup
    args = list(Or(*expr.args).args)
    uniq = []
    for a in args:
        if a not in uniq:
            uniq.append(a)
    args = uniq
    cur = Or(*args)

    while True:
        n = len(args)
        changed = False

        for i in range(n):
            for j in range(i+1, n):
                if satisfiable(And(args[i], args[j])):  # overlap -> not deterministic
                    Vi, Vj = _vars(args[i]), _vars(args[j])
                    cand = list(Vi & Vj) or list(Vi | Vj)  # first intersection, then union
                    for p in cand:
                        new_or = shannon_expansion(Or(*args), p)
                        if new_or != cur:       # check structural progress
                            cur = new_or
                            args = list(cur.args) if isinstance(cur, Or) else [cur]
                            changed = True
                            break
                    break
            if changed:
                break
        if not changed:
            return cur

# ---------------------------
# 2) DNNF -> d-DNNF (determinism on OR)
# ---------------------------
def DNNF2dDNNF(dnnf):
    if isinstance(dnnf, And):
        return And(*(DNNF2dDNNF(a) for a in dnnf.args))
    if isinstance(dnnf, Or):
        kids = [DNNF2dDNNF(a) for a in dnnf.args]
        det = _make_or_deterministic(Or(*kids))
        if isinstance(det, Or):
            return Or(*(DNNF2dDNNF(a) for a in det.args))
        return det
    #print("dnnf: ", dnnf)
    return dnnf  # literal / True / False

# ---------------------------
# 3) d-DNNF -> sd-DNNF (smoothing with Tau)
# ---------------------------
def ddnnf2sdNNF(ddnnf):
    if isinstance(ddnnf, And):
        return And(*(ddnnf2sdNNF(a) for a in ddnnf.args))
    if isinstance(ddnnf, Or):
        kids = [ddnnf2sdNNF(a) for a in ddnnf.args]
        allv = set().union(*(_vars(k) for k in kids))
        smoothed = []
        for k in kids:
            missing = allv - _vars(k)
            kk = k
            for v in missing:
                kk = And(kk, _tau(v))  # keep Tau(v) visible
            smoothed.append(kk)
        return Or(*smoothed)
    #print("ddnnf: ", ddnnf)
    return ddnnf  # literal / True / False

# ---------------------------
# 4) WMC on sd-DNNF (weights to literals)
# ---------------------------
def model_counting_sdnnf(sdNNF_formula, weights):
    """
    Calculate WMC on sd-DNNF:
      - literal: weights[literal]
      - Tau(v):  weights[v] + weights[Not(v)]
      - AND:     products of children
      - OR:      sum of children
    """
    if sdNNF_formula is S.true:
        return 1.0
    if sdNNF_formula is S.false:
        return 0.0

    # literals (a, ~a, ...)
    if sdNNF_formula in weights:
        #print("sdNNF_formula, ", float(weights[sdNNF_formula]))
        return float(weights[sdNNF_formula])

    # Tau(v) inserted from smoothing
    if isinstance(sdNNF_formula, Tau):
        v = sdNNF_formula.v
        return float(weights[v] + weights[Not(v)])

    # AND: product
    if isinstance(sdNNF_formula, And):
        val = 1.0
        for ch in sdNNF_formula.args:
            val *= model_counting_sdnnf(ch, weights)
        return val

    # OR: sum
    if isinstance(sdNNF_formula, Or):
        return sum(model_counting_sdnnf(ch, weights) for ch in sdNNF_formula.args)

    raise ValueError(f"Unmanaged node: {sdNNF_formula}")

# ---------------------------
# 5) Orchestrator End-to-end: F -> sd-DNNF
# ---------------------------
def compile_to_sdDNNF(F,details):
    """
    F -> NNF -> DNNF -> d-DNNF -> sd-DNNF
    (no final simplify to avoid losing the Tau nodes)
    """
    # removes ⇒, ↔ e force NOT to literals
    
    F_nnf  = to_nnf(F, simplify=True)   
    if details:
        print("to_nnf: ", F_nnf)
    time.sleep(2.5)
    stages[0] = ("to NNF", True); handle.update(HTML(render_pipeline(stages)))

    F_dnnf = NNF2DNNF(F_nnf)
    if details:
        print("F_dnnf: ", F_dnnf)
    time.sleep(2)
    stages[1] = ("to DNNF", True); handle.update(HTML(render_pipeline(stages)))
    
    F_dd   = DNNF2dDNNF(F_dnnf)
    if details:
        print("F_dd: ", F_dd)
    time.sleep(2)
    stages[2] = ("to dDNNF", True); handle.update(HTML(render_pipeline(stages)))
    
    F_sdd  = ddnnf2sdNNF(F_dd)
    if details:
        print("F_sdd: ", F_sdd)
    time.sleep(2)
    stages[3] = ("to sdNNF", True); handle.update(HTML(render_pipeline(stages)))
    return F_sdd

For other test cases -> modify the formula and weights section

In [119]:
# ==============================================================
# Exact vs Approximate Model Counting
# ==============================================================

import math, random
from typing import Dict, Tuple
from sympy import symbols
from sympy.logic.boolalg import Boolean, And, Or, Not, Implies, Equivalent

# ------------------ Utilities ------------------

def vars_set(F: Boolean):
    return list(sorted(F.free_symbols, key=lambda s: s.name))

def _vars_set(F):
    if "_vars" in globals() and callable(_vars):
        return set(_vars(F))
    return set(F.free_symbols)

def eval_under(F: Boolean, assign: Dict):
    return bool(F.subs(assign))

def random_assignment(V, rng=random):
    return {v: bool(rng.getrandbits(1)) for v in V}

# ------------------ Approximate (Monte Carlo) ------------------

def required_samples_eps_delta(eps: float, delta: float) -> int:
    if not (0 < eps < 1):   raise ValueError("eps deve essere in (0,1).")
    if not (0 < delta < 1): raise ValueError("delta deve essere in (0,1).")
    return math.ceil((1.0 / (2.0 * eps * eps)) * math.log(2.0 / delta))

def inverse_normal_cdf(p: float) -> float:
    # Acklam approximation
    a = [ -3.969683028665376e+01,  2.209460984245205e+02,
          -2.759285104469687e+02,  1.383577518672690e+02,
          -3.066479806614716e+01,  2.506628277459239e+00 ]
    b = [ -5.447609879822406e+01,  1.615858368580409e+02,
          -1.556989798598866e+02,  6.680131188771972e+01,
          -1.328068155288572e+01 ]
    c = [ -7.784894002430293e-03, -3.223964580411365e-01,
          -2.400758277161838e+00, -2.549732539343734e+00,
           4.374664141464968e+00,  2.938163982698783e+00 ]
    d = [ 7.784695709041462e-03,  3.224671290700398e-01,
          2.445134137142996e+00,  3.754408661907416e+00 ]
    plow, phigh = 0.02425, 1 - 0.02425
    if p < plow:
        q = math.sqrt(-2*math.log(p))
        x = (((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
            ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)
    elif p > phigh:
        q = math.sqrt(-2*math.log(1 - p))
        x = -(((((c[0]*q + c[1])*q + c[2])*q + c[3])*q + c[4])*q + c[5]) / \
              ((((d[0]*q + d[1])*q + d[2])*q + d[3])*q + 1)
    else:
        q = p - 0.5; r = q*q
        x = (((((a[0]*r + a[1])*r + a[2])*r + a[3])*r + a[4])*r + a[5])*q / \
            (((((b[0]*r + b[1])*r + b[2])*r + b[3])*r + b[4])*r + 1)
    e = 0.5 * (1 + math.erf(x / math.sqrt(2))) - p
    u = e * math.sqrt(2*math.pi) * math.exp(x*x/2)
    return x - u / (1 + x*u/2)

def clopper_pearson_ci_simple(k: int, N: int, alpha: float) -> Tuple[float, float]:
    if N == 0: return (0.0, 1.0)
    p_hat = k / N
    z = inverse_normal_cdf(1 - alpha/2)
    se = math.sqrt(p_hat * (1 - p_hat) / N) if 0 < p_hat < 1 else math.sqrt(0.25 / N)
    lo = max(0.0, p_hat - z * se)
    hi = min(1.0, p_hat + z * se)
    return (lo, hi)

def approx_model_count(F: Boolean, eps: float = 0.05, delta: float = 0.05,
                       max_samples: int = None, rng: random.Random = None) -> dict:
    rng = rng or random.Random()
    V = vars_set(F); n = len(V)
    N = required_samples_eps_delta(eps, delta)
    if max_samples is not None: N = min(N, max_samples)
    k = 0
    for _ in range(N):
        A = random_assignment(V, rng)
        if eval_under(F, A): k += 1
    p_hat = k / N
    lo_p, hi_p = clopper_pearson_ci_simple(k, N, alpha=delta)
    factor = 2**n
    return {
        "n": n, "N": N, "k": k,
        "p_hat": p_hat, "p_ci": (lo_p, hi_p),
        "count_hat": p_hat * factor,
        "count_ci": (lo_p * factor, hi_p * factor),
        "eps": eps, "delta": delta,
    }

# ------------------ Test Runner ------------------

def exact_count_sdnnf(F,details):
    sd = compile_to_sdDNNF(F,details)
    V = _vars_set(F)
    weights = {}
    for v in V:
        weights[v] = 1.0
        weights[Not(v)] = 1.0
    return model_counting_sdnnf(sd, weights)


def compare_exact_vs_approx(name: str, F: Boolean,
                            eps=0.05, delta=0.05, seed=42, max_samples=None, n=1,details=True):
    
    print(f"\n=== {name} ===")
    print(f"Formula: {F}")
    exact = exact_count_sdnnf(F,details)

    print(f"Exact  : #SAT = {exact}  over 2^{n} = {2**n}")

    res = approx_model_count(F, eps=eps, delta=delta,
                             max_samples=max_samples, rng=random.Random(seed))
    est = res["count_hat"]; lo, hi = res["count_ci"]
    rel_err = abs(est - exact) / max(1, exact)
    print(f"Approx : #SAT ≈ {est:.3f}  (CI {int((1-delta)*100)}%: [{lo:.3f}, {hi:.3f}])")
    print(f"samples N = {res['N']}, successes k = {res['k']}, p_hat = {res['p_hat']:.6f}")
    print(f"Relative error (point) = {100*rel_err:.2f}%")


# DEMO

In [120]:
# ===========================
# TEST CASE 1 (Example 7.5 dispensa - result = 136)
# ===========================

# Formula: (a & b) | (c & ¬a)
a, b, c = symbols('a b c')
F1 = Or(And(a, b), And(c, Not(a)))
print("Formula:", F1)

# Weights
weights = {
    a: 2,   Not(a): 1,
    b: 5,   Not(b): 3,
    c: 7,   Not(c): 1
}
stages = [("to NNF", False), ("to DNNF", False), ("to dDNNF", False), ("to sdNNF", False), ("Counting", False)]
handle = display(HTML(render_pipeline(stages)), display_id=True)
# Orchestration till sd-DNNF
F1_sdd = compile_to_sdDNNF(F1,details=True)
print("\nsd-DNNF:", F1_sdd)

# WMC calculation with actual weights
wmc_val = model_counting_sdnnf(F1_sdd, weights)
time.sleep(2)
stages[4] = ("Counting", True); handle.update(HTML(render_pipeline(stages)))
print("\nWMC:", wmc_val)

# Results
results = [
    {"name": "F1", "formula":F1, "sd-DNNF": F1_sdd, "exact": 136, "calculated": wmc_val}
]
render_kpi_cards(results)

Formula: (a & b) | (c & ~a)


to_nnf:  (a & b) | (c & ~a)
F_dnnf:  (a & b) | (c & ~a)
F_dd:  (a & b) | (c & ~a)
F_sdd:  (a & b & Tau(c)) | (c & ~a & Tau(b))

sd-DNNF: (a & b & Tau(c)) | (c & ~a & Tau(b))

WMC: 136.0


In [121]:
# ===========================
# TEST CASE 2 — (Exercise 119 dispensa - result = 23)
# ===========================

# Formula: (A ∨ B) → (B ∨ C)
A, B, C = symbols('A B C')
F2 = Implies(Or(A, B), Or(B, C))
print("Formula F2:", F2)

# Literals Weights (from screenshot)
weights = {
    A: 1,  Not(A): 2,
    B: 1,  Not(B): 2,
    C: 1,  Not(C): 2,
}

stages = [("to NNF", False), ("to DNNF", False), ("to dDNNF", False), ("to sdNNF", False), ("Counting", False)]
handle = display(HTML(render_pipeline(stages)), display_id=True)
# Compilazione e WMC
F2_sdd = compile_to_sdDNNF(F2,details=True)
print("\nsd-DNNF:", F2_sdd)

wmc_val = model_counting_sdnnf(F2_sdd, weights)
time.sleep(2)
stages[4] = ("Counting", True); handle.update(HTML(render_pipeline(stages)))
print("\nWMC:", wmc_val)

# Results
results = [
    {"name": "F2", "formula":F2, "sd-DNNF": F2_sdd, "exact": 23, "calculated": wmc_val}
]
render_kpi_cards(results)

Formula F2: Implies(A | B, B | C)


to_nnf:  B | C | (~A & ~B)
F_dnnf:  B | C | ~A
F_dd:  B | (~B & (~A | (A & C)))
F_sdd:  (B & Tau(A) & Tau(C)) | (~B & ((A & C) | (~A & Tau(C))))

sd-DNNF: (B & Tau(A) & Tau(C)) | (~B & ((A & C) | (~A & Tau(C))))

WMC: 23.0


In [123]:
# ==============================================================
# Exact Counting VS Approximate Counting
# ==============================================================

# Vars
A, B, C, D, E = symbols('A B C D E')

F3 = And(A, B)  # #SAT = 1, n=2
F4 = Or(Not(A), Not(B), Not(C))  # #SAT = 7, n=3

# comparison eps=5%

compare_exact_vs_approx("F1: Test 1", F1, eps=0.03, delta=0.05, seed=7,n=1,details=False)
compare_exact_vs_approx("F2: Test 2", F2, eps=0.03, delta=0.05, seed=7,n=2,details=False)
compare_exact_vs_approx("F3: And(A,B)", F3, eps=0.03, delta=0.05, seed=7,n=3,details=False)
compare_exact_vs_approx("F4: 7/8 clause", F4, eps=0.03, delta=0.05, seed=7,n=4,details=False)



=== F1: Test 1 ===
Formula: (a & b) | (c & ~a)
Exact  : #SAT = 4.0  over 2^1 = 2
Approx : #SAT ≈ 4.023  (CI 95%: [3.850, 4.197])
samples N = 2050, successes k = 1031, p_hat = 0.502927
Relative error (point) = 0.59%

=== F2: Test 2 ===
Formula: Implies(A | B, B | C)
Exact  : #SAT = 7.0  over 2^2 = 4
Approx : #SAT ≈ 6.989  (CI 95%: [6.874, 7.104])
samples N = 2050, successes k = 1791, p_hat = 0.873659
Relative error (point) = 0.15%

=== F3: And(A,B) ===
Formula: A & B
Exact  : #SAT = 1.0  over 2^3 = 8
Approx : #SAT ≈ 0.952  (CI 95%: [0.878, 1.026])
samples N = 2050, successes k = 488, p_hat = 0.238049
Relative error (point) = 4.78%

=== F4: 7/8 clause ===
Formula: ~A | ~B | ~C
Exact  : #SAT = 7.0  over 2^4 = 16
Approx : #SAT ≈ 7.013  (CI 95%: [6.899, 7.127])
samples N = 2050, successes k = 1797, p_hat = 0.876585
Relative error (point) = 0.18%
